# Objective:
Enable Azure product groups (PGs) to explore customer feedback and gain insights that will help them understand and prioritize customer needs more effectively using natural language processing and AI tools.

## Problem Use Cases:
- **As a PG PM**, I can collect and present customer feedback on a specific issue to justify investment in addressing it within our product roadmap.
- **As a PG PM**, I need a list of customers who reported a specific challenge to justify the investment in addressing it within our product roadmap.
- **As a PG PM**, I need a list of customers who reported a specific challenge within my product, so I can gather early feedback on my proposed solution.

## Things to research

1. Getting sample feedback from ADX
2. Writing prompts to create user story from the feedback content
3. use embedding on user stories, use vector search to see if it can help in the search of similar feedback

## Getting Sample data from ADX

**As the feedback data might contain customer information, it should not be publicly shared. This notebook does not cover the process of getting access to the relevant ADX cluster.**

These would be the services we would be starting with:

- {"ServiceName": App Service (Web Apps), "FeedbackCount": 1228 }
- {"ServiceName": Azure VMware Solution, "FeedbackCount": 1092}
- {"ServiceName": Azure Kubernetes Service, "FeedbackCount": 746}

The following `kql` query was used to find services with higher count of feedbacks:

```kql
Feedback
| summarize arg_max(PartnerReceivedDate, *) by Id // Get the most recent entry per Id
| extend ServiceName = tostring(ServiceTree.Name) // Extract the 'Name' from ServiceTree
| summarize FeedbackCount = count() by ServiceName // Count the feedback per service
| sort by FeedbackCount desc // Sort by feedback count in descending order
```

For the PoC, we dont need too much data, the following query would be used to sample feedbacks of these services:

```kql
Feedback
| where ServiceTree.Name in ('App Service (Web Apps)', 'Azure VMware Solution', 'Azure Kubernetes Service') 
| summarize arg_max(PartnerReceivedDate, *) by Id
| sample 20 // Take 20 random items
| extend CleanDescription = replace_regex(Description, @"<[^>]*>", '') // Remove HTML tags from Description
| extend CleanWorkaroundDescription = replace_regex(WorkaroundDescription, @"<[^>]*>", '') // Remove HTML tags from WorkaroundDescription
| project Id, PartnerShortName, ServiceTree.Name, Type, Title, Blocking, CleanDescription, WorkaroundAvailable, Priority, Customer.Name,Customer.Tpid, CleanWorkaroundDescription
```

These were selected for the PoC. The exported items are available in `./sample-data/export.csv`

### loading required packages

In [ ]:
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
#r "nuget: DotNetEnv, 2.5.0"

using Azure; 
using Azure.AI.OpenAI;
using DotNetEnv;
using System.IO;
using System.Text.Json; 

## Getting an OpenAI Client

In [ ]:

static string _configurationFile = @"../../configuration/.env";
Env.Load(_configurationFile);

string oAiApiKey = Environment.GetEnvironmentVariable("AOAI_APIKEY") ?? "AOAI_APIKEY not found";
string oAiEndpoint = Environment.GetEnvironmentVariable("AOAI_ENDPOINT") ?? "AOAI_ENDPOINT not found";
string chatCompletionDeploymentName = Environment.GetEnvironmentVariable("CHATCOMPLETION_DEPLOYMENTNAME") ?? "CHATCOMPLETION_DEPLOYMENTNAME not found";
string embeddingDeploymentName = Environment.GetEnvironmentVariable("EMBEDDING_DEPLOYMENTNAME") ?? "EMBEDDING_DEPLOYMENTNAME not found";

AzureKeyCredential azureKeyCredential = new AzureKeyCredential(oAiApiKey);
OpenAIClient openAIClient = new OpenAIClient(new Uri(oAiEndpoint), azureKeyCredential);

Console.WriteLine($"OpenAI Client created: {oAiEndpoint} with: {chatCompletionDeploymentName} and {embeddingDeploymentName} deployments");

### Calling chat completion API

In [ ]:
async Task<string> CallOpenAI(string prompt, string systemMessage)
{
    // Create ChatCompletionsOptions and set up the system and user messages
    ChatCompletionsOptions options = new ChatCompletionsOptions();
    
    // Add system message
    options.Messages.Add(new ChatRequestSystemMessage(systemMessage));
    
    // Add user message (the prompt generated from feedback)
    options.Messages.Add(new ChatRequestUserMessage(prompt));

    // Configure request properties
    options.MaxTokens = 500;
    options.Temperature = 0.7f;
    options.NucleusSamplingFactor = 0.95f;
    options.FrequencyPenalty = 0.0f;
    options.PresencePenalty = 0.0f;
    options.StopSequences.Add("\n"); 
    options.DeploymentName = chatCompletionDeploymentName;
    options.ResponseFormat = ChatCompletionsResponseFormat.Text;

    // Make the API request to get the chat completions
    Response<ChatCompletions> response = await openAIClient.GetChatCompletionsAsync(options);

    // Extract and return the first response from the choices
    ChatCompletions completions = response.Value;
    if (completions.Choices.Count > 0)
    {
        return completions.Choices[0].Message.Content;
    }
    else
    {
        return "No response generated.";
    }
}

### Calling the embeddings API

In [ ]:
async Task<float[]> GetEmbeddingAsync(string textToBeVecorized)
{
    // Prepare the embeddings options with the user story
    EmbeddingsOptions embeddingsOptions = new EmbeddingsOptions(embeddingDeploymentName, new List<string> { textToBeVecorized });
    var modelResponse = await openAIClient.GetEmbeddingsAsync( embeddingsOptions);
    float[] response = modelResponse.Value.Data[0].Embedding.ToArray();
    return response;
}

In [ ]:
#r "nuget: CsvHelper"

In [43]:
// loading the csv feedback record class
# load "../models/CSVFeedbackRecord.cs"


In [ ]:
// Define the system message separately
// var systemMessage = "You are an AI assistant. You generate clear user stories from the provided feedback data.";
// var systemMessage = "You are an AI assistant. You generate clear user stories from the provided feedback data in the format: 'As a persona, I want to do something, so that I can achieve something.'";
var systemMessage = "You are an AI assistant. You generate clear generic user stories in text only, the following format: 'As a [persona], I want to [do something], so that I can [achieve something].' The output should always follow this format without additional styling or formatting. do not include specific customers/partner names as part of the output.";

In [44]:
// loop over a csv file, create a json file with user story and embedding

async Task<bool> GenerateUserStories(string csvFilePath, string jsonFilePath)
{
    using var reader = new StreamReader(csvFilePath);
    using var csv = new CsvHelper.CsvReader(reader, System.Globalization.CultureInfo.InvariantCulture);
    var records = csv.GetRecords<CSVFeedbackRecord>().ToList();
    
    foreach (var record in records)
    {
        // Generate the user story from the feedback record
        record.UserStory = await CallOpenAI(record.ToPrompt(), systemMessage);
        
        // Generate the embedding for the user story
        record.Embedding = await GetEmbeddingAsync(record.UserStory);
        Console.WriteLine($"User story & embedding generated for record with ID: {record.Id}");
    }
    
    // Serialize the records to a JSON file
    var json = JsonSerializer.Serialize(records, new JsonSerializerOptions { WriteIndented = true });
    await File.WriteAllTextAsync(jsonFilePath, json);
    return true;
}

In [ ]:
using System.Globalization;
using CsvHelper;
using System.IO;
using System.Collections.Generic;

// Path to your CSV file (update the path as needed)
var assetPath = "../../sample-data/";
var filePath = $"{assetPath}export.csv";
// Create a list to store the records
var res = await GenerateUserStories(filePath, $"{assetPath}/exportedstories.json");

Console.WriteLine($"JSON file created at: {outputJsonFile}");